Reading the datasets.

In [14]:
import struct
import numpy as np

def load_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
        data = f.read()
        print(f"File size: {len(data)} bytes. Expected: {num_images * rows * cols} bytes.")
        images = np.frombuffer(data, dtype=np.uint8).reshape(num_images, rows, cols)
    return images

def load_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num_labels = struct.unpack(">II", f.read(8))
        data = f.read()
        print(f"File size: {len(data)} bytes. Expected: {num_labels} bytes.")
        labels = np.frombuffer(data, dtype=np.uint8)
    return labels


train_images_path = 'train-images.idx3-ubyte'
train_labels_path = 'train-labels.idx1-ubyte'
test_images_path = 't10k-images.idx3-ubyte'
test_labels_path = 't10k-labels.idx1-ubyte'


train_images = load_images(train_images_path)
train_labels = load_labels(train_labels_path)
test_images = load_images(test_images_path)
test_labels = load_labels(test_labels_path)


print(f"Train Images Shape: {train_images.shape}")
print(f"Train Labels Shape: {train_labels.shape}")
print(f"Test Images Shape: {test_images.shape}")
print(f"Test Labels Shape: {test_labels.shape}")


File size: 47040000 bytes. Expected: 47040000 bytes.
File size: 60000 bytes. Expected: 60000 bytes.
File size: 7840000 bytes. Expected: 7840000 bytes.
File size: 10000 bytes. Expected: 10000 bytes.
Train Images Shape: (60000, 28, 28)
Train Labels Shape: (60000,)
Test Images Shape: (10000, 28, 28)
Test Labels Shape: (10000,)


Preprocess the data and initialize the weights & biases. I found He initialization works best and copied to my project.

In [15]:
import numpy as np
import struct
np.random.seed(42)


def preprocess_images(images):
    # Flatten and normalize the images
    num_images = images.shape[0]
    flattened_images = images.reshape(num_images, -1)  # Flatten to (num_images, 28*28)
    normalized_images = flattened_images / 255.0       # Normalize to range [0, 1]
    return normalized_images
def one_hot_encode(labels, num_classes=10):
    # Ensure labels are integers
    labels = labels.astype(int)
    # Convert labels to one-hot encoded vectors
    one_hot = np.zeros((labels.size, num_classes))
    one_hot[np.arange(labels.size), labels] = 1
    return one_hot

# Preprocess training and test data
train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)
train_labels = one_hot_encode(train_labels)
test_labels = one_hot_encode(test_labels)
def initialize_weights_and_biases_explicit(layer_sizes):
    """
    Initialize weights and biases for a feedforward neural network with explicit access.
    Args:
        layer_sizes: List containing the number of neurons in each layer.
    Returns:
        params: Dictionary containing weights and biases for each layer.
    """
    params = {}
    for i in range(1, len(layer_sizes)):
        # Create keys for weights and biases for layer i
        params[f"W{i}"] = np.random.randn(layer_sizes[i], layer_sizes[i-1]) * np.sqrt(2 / layer_sizes[i-1])  # He initialization
        params[f"b{i}"] = np.zeros((layer_sizes[i],))  # Bias vector shape: (layer_sizes[i],)
    return params

# Define the network architecture
layer_sizes = [784, 128, 64, 10]  # Input layer (784), 2 hidden layers (128, 64), output layer (10)

# Initialize weights and biases
params = initialize_weights_and_biases_explicit(layer_sizes)




For hidden layers I used relu activation function and for the output layer I used softmax. I used cross-entropy as loss function. It takes around 4 minutes to complete depending on the num_epochs value however this was the only way to get consistently 95% or above accuracy. I did not use any external source here since the implementation is very familiar with what we had done in previous hws.

In [3]:
def relu(x):
    return np.maximum(0, x)
def relu_derivative(x):
    return np.where(x > 0, 1, 0)
def softmax(logits):
    logits = logits - np.max(logits)
    exp_scores = np.exp(logits)
    return exp_scores / np.sum(exp_scores)
def compute_loss(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-9))

# hyperparameters for the neural networks
learning_rate = 0.01
num_epochs = 5

for epoch in range(num_epochs):
  # Training of the neural network
  for i in range(0, train_images.shape[0]):
    train_image_vector = train_images[i]
  # Feed forward
    # Input Layer
    z1 = params['W1'] @ train_image_vector + params['b1'] # Multiply the input with the W1 and add the bias, input for the first hidden layer
    # First Hidden Layer
    a1 = relu(z1) # Apply relu on the z1 which will be the output of the HL1.
    z2 = params['W2'] @ a1 + params['b2'] # transform the a1 using the W2 and b2 to create the input for the HL2.
    # Second Hidden Layer
    a2 = relu(z2)
    z3 = params['W3'] @ a2 + params['b3']

    # Output Layer
    a3 = softmax(z3[np.newaxis, :]).flatten()
  # backward propagation
    dz3 = a3 - train_labels[i] # da3 and dz3's respective derivatives simplifies to this
    dW3 = np.outer(dz3, a2)
    db3 = dz3
    da2 = dz3 @ params['W3']

    dz2 = da2 * relu_derivative(z2)
    dW2 = np.outer(dz2, a1)
    db2 = dz2

    da1 = dz2 @ params['W2']
    dz1 = da1 * relu_derivative(z1)
    dW1 = np.outer(dz1, train_image_vector)
    db1 = dz1
  # Update weights and bias accordingly
    params['W1'] -= learning_rate * dW1
    params['W2'] -= learning_rate * dW2
    params['W3'] -= learning_rate * dW3
    params['b1'] -= learning_rate * db1
    params['b2'] -= learning_rate * db2
    params['b3'] -= learning_rate * db3

# Testing neural network
correct_predictions = 0
for i in range(0, test_images.shape[0]):
  test_image_vector = test_images[i]
  z1 = params['W1'] @ test_image_vector + params['b1']
  a1 = relu(z1)
  z2 = params['W2'] @ a1 + params['b2']
  a2 = relu(z2)
  z3 = params['W3'] @ a2 + params['b3']
  a3 = softmax(z3[np.newaxis, :]).flatten()  # Apply softmax
  # Get predicted class (index of max probability)
  predicted_class = np.argmax(a3)

  # Get true class since its one hot encoded
  true_class = np.argmax(test_labels[i])

  # Check if prediction is correct
  if predicted_class == true_class:
      correct_predictions += 1

accuracy = correct_predictions / test_images.shape[0]

print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 96.27%


In [16]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import relu, softmax
# Convert train_images and test_images to tensors
train_images = torch.from_numpy(train_images).float().to(device)
test_images = torch.from_numpy(test_images).float().to(device)
train_labels = torch.from_numpy(train_labels).float().to(device)
test_labels = torch.from_numpy(test_labels).float().to(device)


In [20]:



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
train_images = train_images.reshape( -1,28, 28)
test_images = test_images.reshape( -1,28, 28)

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0)  # 1 input channel, 16 output channels
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 10 * 10, 48)
        self.fc2 = nn.Linear(48, 10)  # Output for 10 classes

    def forward(self, x):
        x = relu(self.conv1(x))
        x = self.pool1(x)
        x = relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 32 * 10 * 10)  # Flatten
        x = relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, loss function, and optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 5
batch_size = 32
num_batches = train_images.shape[0] // batch_size

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(num_batches):
        # Get mini-batch
        batch_images = train_images[i * batch_size:(i + 1) * batch_size].unsqueeze(1)
        batch_labels = train_labels[i * batch_size:(i + 1) * batch_size]

        # Forward pass
        outputs = model(batch_images)
        loss = criterion(outputs, torch.argmax(batch_labels, dim=1))
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

# Testing loop
model.eval()
correct_predictions = 0
with torch.no_grad():
    for i in range(test_images.shape[0]):
        test_image = test_images[i].unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
        test_label = torch.argmax(test_labels[i])
        output = model(test_image)
        predicted_class = torch.argmax(output)
        if predicted_class == test_label:
            correct_predictions += 1

accuracy = correct_predictions / len(test_images)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Using device: cuda
Epoch 1/5, Loss: 411.7756
Epoch 2/5, Loss: 237.4963
Epoch 3/5, Loss: 220.8316
Epoch 4/5, Loss: 212.0178
Epoch 5/5, Loss: 202.2744
Test Accuracy: 96.27%


OPTIONAL=> MY IMPLEMENTATION OF CNN

In [ ]:
import math
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def conv2d_no_channel(input_image, filter, stride, padding):
    """
    Perform 2D convolution on an input image without channel dimension.
    """
    # Add zero-padding to the input image
    # if padding > 0:
    #     input_image = np.pad(input_image, ((padding, padding), (padding, padding)), mode='constant')

    # Get input and filter dimensions
    H, W = input_image.shape
    kH, kW = filter.shape

    # Calculate output dimensions
    H_out = (H - kH) // stride + 1
    W_out = (W - kW) // stride + 1

    # Initialize output feature map
    feature_map = np.zeros((H_out, W_out))

    # Perform convolution
    for i in range(0, H_out):
        for j in range(0, W_out):
            region = input_image[i * stride:i * stride + kH, j * stride:j * stride + kW]
            feature_map[i, j] = np.sum(region * filter)

    return feature_map
def max_pooling(feature_map, pool_size, stride):

    H, W = feature_map.shape
    H_out = (H - pool_size) // stride + 1
    W_out = (W - pool_size) // stride + 1

    pooled_map = np.zeros((H_out, W_out))


    for i in range(0, H_out):
        for j in range(0, W_out):
            region = feature_map[i * stride:i * stride + pool_size, j * stride:j * stride + pool_size]
            pooled_map[i, j] = np.max(region)

    return pooled_map

def softmax(logits):
    logits = logits - np.max(logits)  # Numerical stability
    exp_scores = np.exp(logits)
    return exp_scores / np.sum(exp_scores)

def dense_layer(input_vector, weights, bias):
    return np.dot(weights, input_vector) + bias

# initialize the parameters here
# This is crucial because I want to get 95% accuracy so I have to change these values frequently.
convstride1 = 1  # Smaller stride to preserve spatial details
convpadding1 = 0  # No padding
poolingsize1 = 2  # Standard pooling size
poolingstride1 = 2  # Non-overlapping pooling

convstride2 = 1  # Smaller stride for better feature extraction
convpadding2 = 0  # No padding
poolingsize2 = 2  # Standard pooling size
poolingstride2 = 1  # Overlapping pooling

filter1size = 3  # Larger filter to capture more patterns
filter2size = 3  # Larger filter in the second layer

# Calculate flattened_vector_size dynamically
flattened_vector_size = 10 * 10  # Example output size of the second pooling layer (10x10 = 100)



learning_rate = 0.01
# Initialize the weights & biases and the kernels
filter1_shape = (filter1size, filter1size)
filter2_shape = (filter2size, filter2size)

# He initialization for convolutional kernels
filter1 = np.random.randn(*filter1_shape) * np.sqrt(2 / np.prod(filter1_shape))
filter2 = np.random.randn(*filter2_shape) * np.sqrt(2 / np.prod(filter2_shape))


# Fully Connected Layer 1 (Flattened → Hidden Layer)
w1 = np.random.randn(48, flattened_vector_size) * np.sqrt(2 / flattened_vector_size)
b1 = np.zeros(48)

w2 = np.random.randn(10, 48) * np.sqrt(2 / 48)
b2 = np.zeros(10)




# Training loop
for epoch in range(5):
  total_loss=0
  for i in range(train_images.shape[0]):

      train_image_matrix = train_images[i]
      #train_image_matrix = Timage

      # --- Forward Pass ---
      # First convolutional layer

      feature_map1 = conv2d_no_channel(train_image_matrix, filter1, stride = convstride1, padding = convpadding1)

      feature_map1_activated = relu(feature_map1)

      # First pooling layer
      pooled_map1 = max_pooling(feature_map1_activated, pool_size=poolingsize1, stride=poolingstride1)

      # Second convolutional layer
      feature_map2 = conv2d_no_channel(pooled_map1, filter2, stride=convstride2, padding=convpadding2)

      feature_map2_activated = relu(feature_map2)

      # Second pooling layer
      pooled_map2 = max_pooling(feature_map2_activated, pool_size=poolingsize1, stride=poolingstride2)

      # Flatten the output for the neural network
      flattened_vector = pooled_map2.flatten()


      # Fully connected layer 1
      fc1_output = dense_layer(flattened_vector, w1, b1)
      fc1_activated = relu(fc1_output)

      # Fully connected layer 2 (Output)
      logits = dense_layer(fc1_activated, w2, b2)
      output_probs = softmax(logits)
       # Calculate loss
      y_true = train_labels[i]
      loss = -np.sum(y_true * np.log(np.clip(output_probs, 1e-15, 1.0)))

      total_loss += loss  # Accumulate loss
      # --- Backpropagation ---
      y_true = train_labels[i]
      dz2 = output_probs - y_true

      dw2 = np.outer(dz2, fc1_activated)
      db2 = dz2
      da1 = np.dot(w2.T, dz2)
      dz1 = da1 * relu_derivative(fc1_output)
      dw1 = np.outer(dz1, flattened_vector)
      db1 = dz1
      # layer's backpropagation
      d_input = np.dot(w1.T, dz1)
      side_length = int(math.sqrt(flattened_vector.size))
      d_input = d_input.reshape(side_length, side_length) # which is equal to
      d_pooled_map2 = d_input
      #print(d_pooled_map2.shape)
      # max pooling 2 backpropagation
      d_feature_map2_activated = np.zeros_like(feature_map2_activated)
      for pl_row in range(0, d_pooled_map2.shape[0]):
        for pl_column in range(0, d_pooled_map2.shape[1]):
          # get the region from feature map2
          start_row = pl_row * poolingstride2 # multiply it with stride
          start_column = pl_column * poolingstride2
          end_row = start_row + poolingsize2 # add the pool size to get the regions other end
          end_column = start_column + poolingsize2
          region = feature_map2_activated[start_row:end_row, start_column: end_column] # get the nxn column where we applied max pooling which corresponds to d_inputs[pl_row, pl_column]
          #print(region.shape)
          max_index = np.unravel_index(np.argmax(region), region.shape) # get the max index from to region
          max_pos = (start_row + max_index[0], start_column + max_index[1]) # apply to the current indexes
          # all we need to do assign the loss
          d_feature_map2_activated[max_pos] += d_pooled_map2[pl_row, pl_column]
      d_feature_map2 = relu_derivative(d_feature_map2_activated)

      # convolution layer 2 backpropagation
      d_pooled_map1 = np.zeros_like(pooled_map1)
      d_filter2 = np.zeros_like(filter2)
      for df2_row in range(0, d_feature_map2.shape[0]):
        for df2_column in range(0, d_feature_map2.shape[1]):
          start_row = df2_row * convstride2 # multiply with the stride
          start_column = df2_column * convstride2
          region = pooled_map1[start_row: start_row+filter2size, start_column: start_column+filter2size] # +'s are kernel size, e.g 2x2
          d_filter2+= region * d_feature_map2[df2_row, df2_column]
          d_pooled_map1[start_row: start_row+filter2size, start_column: start_column+filter2size] += filter2 * d_feature_map2[df2_row, df2_column]

      # max pooling 1 backpropagation
      d_feature_map1_activated = np.zeros_like(feature_map1_activated)
      for pl_row in range(0, d_pooled_map1.shape[0]):
        for pl_column in range(0, d_pooled_map1.shape[1]):
          start_row = pl_row * poolingstride1 # multiply it with stride
          start_column = pl_column * poolingstride1
          end_row = start_row + poolingsize1 # add the pool size to get the regions other end
          end_column = start_column + poolingsize1
          region = feature_map1_activated[start_row:end_row, start_column: end_column]
          max_index = np.unravel_index(np.argmax(region), region.shape)
          max_pos = (start_row + max_index[0], start_column + max_index[1])
          d_feature_map1_activated[max_pos] += d_pooled_map1[pl_row, pl_column]
      d_feature_map1 = relu_derivative(d_feature_map1_activated)

      # convolution layer 1 backpropagation
      d_filter1 = np.zeros_like(filter1)
      for df1_row in range(0, d_feature_map1.shape[0]):
        for df1_column in range(0, d_feature_map1.shape[1]):
          start_row = df1_row * convstride1
          start_column = df1_column * convstride1
          region = train_image_matrix[start_row: start_row+filter1size, start_column:start_column+filter1size]
          d_filter1+= region * d_feature_map1[df1_row, df1_column]

      # --- Parameter Updates ---
      w2 -= learning_rate * dw2
      b2 -= learning_rate * db2
      w1 -= learning_rate * dw1
      b1 -= learning_rate * db1
      filter2 -= learning_rate * d_filter2
      filter1 -= learning_rate * d_filter1
  # Print loss after each epoch
  print(f"Epoch {epoch + 1}/{5}, Loss: {total_loss:.4f}")

# TESTING
correct_predictions = 0  # Counter for correct predictions

for i in range(len(test_images)):
    test_image_matrix = test_images[i]

   # First convolutional & max pooling layers
    feature_map1 = conv2d_no_channel(test_image_matrix, filter1, stride=convstride1, padding=convpadding1)
    feature_map1_activated = relu(feature_map1)
    pooled_map1 = max_pooling(feature_map1_activated, pool_size=poolingsize1, stride=poolingstride1)

    # Second convolutional & max pooling layers
    feature_map2 = conv2d_no_channel(pooled_map1, filter2, stride=convstride2, padding=convpadding2)
    feature_map2_activated = relu(feature_map2)
    pooled_map2 = max_pooling(feature_map2_activated, pool_size=poolingsize2, stride=poolingstride2)

    # Flatten the output for the neural network
    flattened_vector = pooled_map2.flatten()

    # Fully connected layer1
    fc1_output = dense_layer(flattened_vector, w1, b1)
    fc1_activated = relu(fc1_output)

    # Fully connected layer2 (Output layer)
    logits = dense_layer(fc1_activated, w2, b2)
    output_probs = softmax(logits)

    # Predicted class
    predicted_class = np.argmax(output_probs)
    true_class = np.argmax(test_labels[i])  # True label from test set

    # Check if the prediction is correct
    if predicted_class == true_class:
        correct_predictions += 1

# --- Calculate Accuracy ---
accuracy = correct_predictions / len(test_images)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
